In [1]:
'''
changed (comment out) http://localhost:8888/edit/workplace/mxnet/python/mxnet/module/base_module.py
line 144 411 412
'''


import mxnet as mx
import numpy as np
out_dim = 6
compute_size = 128
D1 = mx.sym.Variable('data1')
S1 = mx.sym.Variable('s1',init = mx.init.Plusminusone(),shape = (1,20))
H1 = mx.sym.Variable('h1',init = mx.init.Index(out_dim),shape = (1,20))
D2 = mx.sym.Variable('data2')
S2 = mx.sym.Variable('s2',init = mx.init.Plusminusone(),shape = (1,20))
H2 = mx.sym.Variable('h2',init = mx.init.Index(out_dim),shape = (1,20))

cs1 = mx.sym.CountSketch( data = D1,s=S1, h = H1 ,name='cs1',out_dim = out_dim) 
cs2 = mx.sym.CountSketch( data = D2,s=S2, h = H2 ,name='cs2',out_dim = out_dim) 
fft1 = mx.sym.FFT(data = cs1, name='fft1', compute_size = compute_size) 
fft2 = mx.sym.FFT(data = cs2, name='fft2', compute_size = compute_size) 
c = fft1 * fft2
out = mx.sym.IFFT(data = c, name='ifft', compute_size = compute_size) 

mod = mx.mod.Module(symbol=out, 
                    context=mx.gpu(0),
                    data_names=['data1','data2'],
                    label_names = None
                    )

data = mx.io.NDArrayIter({'data1':np.random.uniform(-1, 1, (10,20)),'data2':np.random.uniform(-1, 1, (10,20))},batch_size=10,label_name=None)

from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

mod.bind(data_shapes=[('data1', (10, 20)),('data2', (10, 20))])
mod.init_params()
print mod.label_shapes
print mod.get_params()[0]['s1'].asnumpy()
print mod.get_params()[0]['h1'].asnumpy()
#mod.forward(Batch(data))
#mod.forward(Batch(data))
mod.fit(data, num_epoch=1)
print mod.get_outputs()[0].asnumpy()

None
[[-1. -1. -1. -1. -1. -1.  1.  1.  1.  1. -1.  1. -1. -1.  1.  1.  1. -1.
  -1.  1.]]
[[ 5.  0.  5.  0.  3.  5.  2.  0.  2.  0.  4.  2.  4.  5.  4.  2.  4.  0.
   4.  0.]]
[[ 28.20666504   2.5480504    5.23099422 -19.15708923   5.23099422
    2.5480504 ]
 [ 22.68682671   6.10175562   1.3749795  -19.89404488   1.3749795
    6.10175562]
 [ -7.58494759   9.37327385  -6.09039211  16.31090927  -6.09039211
    9.37327385]
 [-11.65919781   3.3057375  -27.06985474 -18.99271011 -27.06985474
    3.30573654]
 [  5.03262043   1.06160045  -4.891366    -7.159832    -4.891366
    1.06160045]
 [-26.49144745   0.95978546 -10.09792519   3.39095688 -10.09792519
    0.95978594]
 [ 18.80475426  -3.78564358   8.0124054   -8.31191063   8.01240635
   -3.7856431 ]
 [ 15.93953323  -4.49252319   7.64351606   9.00363159   7.64351654
   -4.49252272]
 [ -0.79262215   0.14079565  -0.27017492   0.53937823  -0.27017486
    0.14079569]
 [  3.02468252  -2.98793697  -0.77909338   1.07406473  -0.7790935
   -2.9879369